In [6]:
import os 
import sys
import argparse
from argparse import Namespace

import numpy as np
import time
import torch
import torch
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import torchvision
from PIL import Image
import torchvision.transforms as T

import src.misc.dist as dist 
from src.core import YAMLConfig 
from src.solver import TASKS
from src.zoo.rtdetr.box_ops import box_cxcywh_to_xyxy as rtdetr_box_cxcywh_to_xyxy

np.random.seed(0)
print(torch.cuda.is_available())
print(torch.__version__)

True
2.5.1+cu121


In [7]:
args = Namespace(
                config='configs/rtdetr/rtdetr_r50vd_6x_coco_customed.yml', 
                # config='configs/rtdetr/rtdetr_r50vd_6x_coco.yml', 
                resume=None, 
                tuning='rtdetr_r50vd_6x_coco_from_paddle.pth', 
                test_only=True, 
                amp=True, 
                seed=0)

cfg = YAMLConfig(
    args.config,
    resume=args.resume, 
    use_amp=args.amp,
    tuning=args.tuning
)

solver = TASKS[cfg.yaml_cfg['task']](cfg)
solver.setup()
solver.eval()
solver.model.eval()
1

Load PResNet50 state_dict
Tuning checkpoint from rtdetr_r50vd_6x_coco_from_paddle.pth


d:\Hust master thesis\codes\rtdetr_pytorch\src\solver\solver.py:148: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(path, map_location='cpu')


Load model.state_dict, {'missed': [], 'unmatched': []}
Tuning checkpoint from rtdetr_r50vd_6x_coco_from_paddle.pth


d:\Hust master thesis\codes\rtdetr_pytorch\src\core\yaml_utils.py:148: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  return cls(**cls_kwargs)


Load model.state_dict, {'missed': [], 'unmatched': []}
loading annotations into memory...


c:\Users\ABC\.conda\envs\py310\lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Done (t=0.93s)
creating index...
index created!


1

In [14]:
embed_dim = 256  # Kích thước embedding
num_heads = 8    # Số lượng head
bs = 4   # Kích thước batch
seq_len_q = 300   # Độ dài chuỗi query
seq_len_kv = 300  # Độ dài chuỗi key/value
dropout = 0.1    # Tỷ lệ dropout

query = torch.ones(bs, seq_len_q, embed_dim, device=solver.device)  # (N, L, E)
key = torch.ones(bs, seq_len_kv, embed_dim, device=solver.device)  # (N, S, E)
value = torch.ones(bs, seq_len_kv, embed_dim, device=solver.device)  # (N, S, E)

# Tạo mask cho query
query_mask = torch.zeros((bs, num_heads, seq_len_q), dtype=torch.bool, device=solver.device) # [bs, nhead, num_queries]
query_mask[..., 150:] = True

query_attn_mask = query_mask.unsqueeze(-1) | query_mask.unsqueeze(-2) # [bs, nhead, num_queries, num_queries]
query_attn_mask = query_attn_mask.reshape(-1, seq_len_q, seq_len_q)  # [bs * nhead, num_queries, num_queries]

query_attn_mask = None

In [9]:
from src.zoo.rtdetr.customed_decoder3 import FlashMultiheadAttention


flash_attn = FlashMultiheadAttention(
    embed_dim=embed_dim,
    num_heads=num_heads,
    dropout=dropout,
    bias=True,
    batch_first=True,
    device=solver.device,
    dtype=torch.float32
)

In [17]:
total_time = 0
iterations = 1000
# Thực hiện forward pass và đo thời gian
for _ in range(iterations):
    torch.cuda.synchronize()  # Đảm bảo GPU đã sẵn sàng
    start_time = time.time()
    # Forward pass
    output, _ = flash_attn(
        query=query,
        key=key,
        value=value,
        key_padding_mask=None,  # Có thể thêm key_padding_mask nếu cần
        need_weights=False,
        attn_mask=query_attn_mask,  # Sử dụng mask đã tạo
        is_causal=False  # Đặt False vì đã cung cấp mask
    )
    torch.cuda.synchronize()  # Đảm bảo GPU đã hoàn thành
    end_time = time.time()
    total_time += (end_time - start_time)
    
avg_time = total_time / iterations
print(f"Average forward pass time: {avg_time:.6f} seconds")


Average forward pass time: 0.001153 seconds
